<a href="https://colab.research.google.com/github/jzinnegger/differential-ml/blob/main/Illustrations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Illustration of pathwise learning with  differentials

The notebook illustrates the pathwise learning of the fair value and delta (as a function of the underlying stock price) in the Black Scholes setup. The example is based on the work of Brian Huge and Antoine Savine (see Working paper: https://arxiv.org/abs/2005.02347 and GitHub: https://github.com/differential-machine-learning).

The samples of the initial stock price $S_1$ at $T_1$ are drawn from a uniform distribution. In the original example the initial state space at $T_1$ can be seen as a time slice of the (simulated) process starting at $T_0$. The time slice approach is convenient to the derive a consitent state space for a higher dimensional market model where all paths follow the same underlying model and calibration.

The twin neural network is indepedently implemented in Keras/Tensorflow2.
The implementation of the model is provided in [Github: jzinnegger/differential-ml](https://github.com/jzinnegger/differential-ml/)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import time
import datetime             

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm.keras import TqdmCallback

from scipy.stats import norm

import pathlib
import shutil
import tempfile

tf.keras.backend.set_floatx('float32') # default
real_type = tf.float32

In [ ]:
# clone git
import os
os.chdir("/content")
!rm -rf differential-ml
!git clone --depth=1 https://github.com/jzinnegger/differential-ml.git
os.chdir("./differential-ml")
from my_python.models import *

### Generate training data of pathwise values and deltas

In [ ]:
import numpy as np
from scipy.stats import norm  
# helper analytics    
def bsPrice(spot, strike, vol, T):
    d1 = (np.log(spot/strike) + vol * vol * T) / vol / np.sqrt(T)
    d2 = d1 - vol * np.sqrt(T)
    return spot * norm.cdf(d1) - strike * norm.cdf(d2)

def bsDelta(spot, strike, vol, T):
    d1 = (np.log(spot/strike) + vol * vol * T) / vol / np.sqrt(T)
    return norm.cdf(d1)

def bsVega(spot, strike, vol, T):
    d1 = (np.log(spot/strike) + vol * vol * T) / vol / np.sqrt(T)
    return spot * np.sqrt(T) * norm.pdf(d1)
#
    
# main class
class BlackScholes:
    
    def __init__(self, 
                 vol=0.2,
                 T1=1, 
                 T2=2, 
                 K=1.10,
                 volMult=1.5,
                 lower=0.35,
                 upper=1.65):
        
        self.spot = 1
        self.vol = vol
        self.T1 = T1
        self.T2 = T2
        self.K = K
        self.volMult = volMult
                        
    # training set: returns S1 (mx1), C2 (mx1) and dC2/dS1 (mx1)
    def trainingSet(self, m,  anti=True, seed=None):
    
        np.random.seed(seed)
        
        # 2 sets of normal returns
        returns = np.random.normal(size=[m, 2])

        # SDE
        vol0 = self.vol * self.volMult
        R1 = np.exp(-0.5*vol0*vol0*self.T1 + vol0*np.sqrt(self.T1)*returns[:,0])

        R2 = np.exp(-0.5*self.vol*self.vol*(self.T2-self.T1) \
                    + self.vol*np.sqrt(self.T2-self.T1)*returns[:,1])
        S1 = self.spot * R1
        S2 = S1 * R2 

        # payoff
        pay = np.maximum(0, S2 - self.K)
        
        # two antithetic paths
        if anti:
            
            R2a = np.exp(-0.5*self.vol*self.vol*(self.T2-self.T1) \
                    - self.vol*np.sqrt(self.T2-self.T1)*returns[:,1])
            S2a = S1 * R2a             
            paya = np.maximum(0, S2a - self.K)
            
            X = S1
            Y = 0.5 * (pay + paya)
    
            # differentials
            Z1 =  np.where(S2 > self.K, R2, 0.0).reshape((-1,1)) 
            Z2 =  np.where(S2a > self.K, R2a, 0.0).reshape((-1,1)) 
            Z = 0.5 * (Z1 + Z2)
                    
        # standard
        else:
        
            X = S1
            Y = pay
            
            # differentials
            Z =  np.where(S2 > self.K, R2, 0.0).reshape((-1,1)) 
        
        return X.reshape([-1,1]), Y.reshape([-1,1]), Z.reshape([-1,1])
    
    def trainingSetUniformS1(self, m, lower=0.2, upper=2.0, anti=True, seed=None):

        np.random.seed(seed)

        # 1 set of uniform samples in the one-dim parameter space for S1=S1(R1)
        S1 = np.random.uniform(lower,upper,m) * self.spot
        
        # 2 sets of normal returns, only R2 required
        returns = np.random.normal(size=[m, 1])

        # SDE
        R2 = np.exp(-0.5*self.vol*self.vol*(self.T2-self.T1) \
                    + self.vol*np.sqrt(self.T2-self.T1)*returns[:,0])
        # S1 = self.spot * R1
        S2 = S1 * R2 

        # payoff
        pay = np.maximum(0, S2 - self.K)
        
        # two antithetic paths
        if anti:
            
            R2a = np.exp(-0.5*self.vol*self.vol*(self.T2-self.T1) \
                    - self.vol*np.sqrt(self.T2-self.T1)*returns[:,0])
            S2a = S1 * R2a             
            paya = np.maximum(0, S2a - self.K)
            
            X = S1
            Y = 0.5 * (pay + paya)
    
            # differentials
            Z1 =  np.where(S2 > self.K, R2, 0.0).reshape((-1,1)) 
            Z2 =  np.where(S2a > self.K, R2a, 0.0).reshape((-1,1)) 
            Z = 0.5 * (Z1 + Z2)
                    
        # standard
        else:
        
            X = S1
            Y = pay
            
            # differentials
            Z =  np.where(S2 > self.K, R2, 0.0).reshape((-1,1)) 
        
        return X.reshape([-1,1]), Y.reshape([-1,1]), Z.reshape([-1,1])

    # test set: returns a grid of uniform spots 
    # with corresponding ground true prices, deltas and vegas
    def testSet(self, lower=0.35, upper=1.65, num=100, seed=None):
        
        spots = np.linspace(lower, upper, num).reshape((-1, 1))
        # compute prices, deltas and vegas
        prices = bsPrice(spots, self.K, self.vol, self.T2 - self.T1).reshape((-1, 1))
        deltas = bsDelta(spots, self.K, self.vol, self.T2 - self.T1).reshape((-1, 1))
        vegas = bsVega(spots, self.K, self.vol, self.T2 - self.T1).reshape((-1, 1))
        return spots, spots, prices, deltas, vegas

### Training values and deltas jointly with the twin network

Equal weighting of values and differentials in the training (alpha = 0.5)

In [ ]:
weightSeed = np.random.randint(0, 10000)
log_dir = "tensorboard_logs/"
#!mkdir {log_dir}f"illu_bs"
nTest = 4096
sizes = [4096, 4096*2, 4096*4]  
generator = BlackScholes()
x_train, y_train, dydx_train = generator.trainingSetUniformS1(max(sizes), seed=None, anti=False)
x_true, x_axis, y_true, dydx_true, vegas = generator.testSet(num=nTest, lower = 0.2, upper=2.0, seed=None)
size=sizes[2]

In [ ]:
prep_layer, scaled_MSE =  preprocess_data(x_train, y_train, dydx_train, prep_type='Normalisation')
model = build_and_compile_model(prep_layer.output_n(),
                                    get_model_autodiff,
                                    scaled_MSE, 
                                    lr_schedule = lr_inv_time_decay,  #  lr_warmup, lr_inv_time_decay
                                    alpha = 0.5
                                )
history = train_model(model,
                        prep_layer,
                        f"illu_bs", 
                        x_train[0:size,:], 
                        y_train[0:size,:], 
                        dydx_train[0:size,:], 
                        epochs=EPOCHS,
                        x_true = x_true, 
                        y_true = y_true, 
                        dydx_true = dydx_true) 
y_pred, dydx_pred = predict_unscaled(model, prep_layer, x_true)



### Illustration

In [ ]:
fig, ax = plt.subplots(1, 2, sharex='row',  figsize=(18,7), squeeze=False)
sample_path = 300
# Fair value
ax[0,0].plot(x_train * 100, y_train, 'c.', markersize=1.5, markerfacecolor='white', label='Training data: Pathwise values')
ax[0,0].plot(x_axis*100, y_pred, 'b.', markersize=0.5, markerfacecolor='white', label='Prediction: Expected values')
ax[0,0].plot(x_train[sample_path] * 100, y_train[sample_path], 'ro', markersize=7, markerfacecolor='r', label=f"Training data: Path {sample_path:d}")
# ax[0,0].plot(x_axis*100, y_true, 'r.', markersize=0.5, markerfacecolor='white', label='True values')

#ax[0,0].set_xlim(0.60*100, 1.65*100)
ax[0,0].set_ylim(-0.01, 0.6)
ax[0,0].set_title("Fair value")
ax[0,0].legend(loc='upper left')
ax[0,0].set_xlabel('Initial stock price at $T_1$')
# Deltas
deltidx=0
ax[0,1].plot(x_train * 100, dydx_train[:,deltidx], 'c.', markersize=1.5, markerfacecolor='white', label='Training data: Pathwise deltas')
ax[0,1].plot(x_axis*100, dydx_pred[:,deltidx], 'b.', markersize=0.5, markerfacecolor='white', label='Prediction: Expected deltas')
ax[0,1].plot(x_train[sample_path] * 100, dydx_train[sample_path,deltidx], 'ro', markersize=7,  markerfacecolor='r', label=f"Training data: Path {sample_path:d}")
# ax[0,1].plot(x_axis*100, dydx_true[:,deltidx], 'r.', markersize=0.5, markerfacecolor='white', label='True deltas')
ax[0,1].set_ylim(-0.05, 2)
ax[0,1].set_title("Delta")
ax[0,1].legend(loc='upper left')
ax[0,1].set_xlabel('Initial stock price at $T_1$')
plt.xlim(0.6*100, 1.65*100)
plt.savefig('illustration_1.png')
plt.show()

### Train net on values only.

Alpha = 1

In [ ]:
prep_layer, scaled_MSE =  preprocess_data(x_train, y_train, dydx_train, prep_type='NoNormalisation')
model = build_and_compile_model(prep_layer.output_n(),
                                    get_model_autodiff,
                                    scaled_MSE, 
                                    lr_schedule = lr_inv_time_decay,  #  lr_warmup, lr_inv_time_decay
                                    alpha = 1)
history = train_model(model,
                        prep_layer,
                        f"illu_bs", 
                        x_train[0:size,:], 
                        y_train[0:size,:], 
                        dydx_train[0:size,:], 
                        epochs=EPOCHS,
                        x_true = x_true, 
                        y_true = y_true, 
                        dydx_true = dydx_true) 
y_pred, dydx_pred = predict_unscaled(model, prep_layer, x_true)



In [ ]:
fig, ax = plt.subplots(1, 2, sharex='row',  figsize=(18,7), squeeze=False)
# Fair value
ax[0,0].plot(x_train * 100, y_train, 'c.', markersize=1.5, markerfacecolor='white', label='Training data: Pathwise values')
ax[0,0].plot(x_axis*100, y_pred, 'b.', markersize=0.5, markerfacecolor='white', label='Prediction: Expected values')
ax[0,0].plot(x_train[sample_path] * 100, y_train[sample_path], 'ro', markersize=7, markerfacecolor='r', label=f"Training data: Path {sample_path:d}")
# ax[0,0].plot(x_axis*100, y_true, 'r.', markersize=0.5, markerfacecolor='white', label='True values')

#ax[0,0].set_xlim(0.60*100, 1.65*100)
ax[0,0].set_ylim(-0.01, 0.6)
ax[0,0].set_title("Fair value")
ax[0,0].legend(loc='upper left')
ax[0,0].set_xlabel('Initial stock price at $T_1$')
# Deltas
deltidx=0
# ax[0,1].plot(x_train * 100, dydx_train[:,deltidx], 'c.', markersize=1.5, markerfacecolor='white', label='Training data: Pathwise deltas')
ax[0,1].plot(x_axis*100, dydx_pred[:,deltidx], 'b.', markersize=0.5, markerfacecolor='white', label='Prediction: Expected deltas')
# ax[0,1].plot(x_axis*100, dydx_true[:,deltidx], 'r.', markersize=0.5, markerfacecolor='white', label='True deltas')
ax[0,1].set_ylim(-0.05, 2)
ax[0,1].set_title("Delta")
ax[0,1].legend(loc='upper left')
ax[0,1].set_xlabel('Initial stock price at $T_1$')
plt.xlim(0.6*100, 1.65*100)
plt.savefig('illustration_2.png')
plt.show()